In [63]:
import numpy as np
import matplotlib.pyplot as plt
import os
from glob import glob
import skimage as ski
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import pandas as pd
from sklearn.model_selection import KFold
from skimage import transform as ski_transform
from torchvision.transforms import ToTensor, Normalize, Compose, RandomRotation
from torchvision import models
import torchvision.transforms.functional as FT
from torcheval.metrics import functional as FM
from torchinfo import summary


In [112]:
transforms = Compose([
    ToTensor(),
    RandomRotation(10),
    Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225])
])

def joint_random_rotation(image, label, degrees):
    # Get a random angle from the range [-degrees, degrees]
    angle = RandomRotation.get_params([-degrees, degrees])
    
    # Rotate the image using bilinear interpolation
    image = FT.rotate(image, angle, interpolation=Image.BILINEAR)
    
    # Rotate the label using nearest neighbor to avoid interpolating label values
    label = FT.rotate(label, angle, interpolation=Image.NEAREST)

    return image, label

def joint_random_horizontal_flip(image, label):
    if torch.rand(1) < 0.5:
        image = FT.hflip(image)
        label = FT.hflip(label)
    return image, label

def joint_random_vertical_flip(image, label):
    if torch.rand(1) < 0.5:
        image = FT.vflip(image)
        label = FT.vflip(label)

    return image, label

def joint_random_crop(image, label, output_size):
    # Get the size of the image
    w, h = image.size

    # Get the target size
    th, tw = output_size

    # Randomly crop the image
    i = torch.randint(0, h - th + 1, (1,))
    j = torch.randint(0, w - tw + 1, (1,))
    image = FT.crop(image, i.item(), j.item(), th, tw)
    
    # Crop the label using the same parameters
    label = FT.crop(label, i.item(), j.item(), th, tw)

    return image, label

In [113]:
BATCH_SIZE = [16]
EPOCHS = 200
LEARNING_RATE = [0.0001]#,0.0001,0.00001]
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
RESIZE_DIM = (256, 256)
fold_number = [0,1,2,3,4]
repeats = 3
val_step = 1 # Every 3 epoch carry out validation
seed = 42
np.random.seed(seed)
torch.manual_seed(seed)

In [114]:
## Provide training path and test_path
train_path = r'/home/biomedialab/Desktop/Codes/Assignments/MLDS/Assignment_2/Q2/mlds_assignmet_2_ml_dl/Dataset/train'
test_path = r'/home/biomedialab/Desktop/Codes/Assignments/MLDS/Assignment_2/Q2/mlds_assignmet_2_ml_dl/Dataset/test'

train_csv_path = r'/home/biomedialab/Desktop/Codes/Assignments/MLDS/Assignment_2/Q2/mlds_assignmet_2_ml_dl/train.csv'
test_csv_path = r'/home/biomedialab/Desktop/Codes/Assignments/MLDS/Assignment_2/Q2/mlds_assignmet_2_ml_dl/test.csv'

In [115]:
"""
id : Name of the image
Binary prediction
    Healthy = 0
    Diseased = 1
segmentation prediction
    Probably the id of the diseased pixel
"""
train_csv_data = pd.read_csv(train_csv_path)
#Sort train_csv_data by id
train_csv_data = train_csv_data.sort_values(by='id')
train_csv_data.head(10)


,id,binary_pred,segmentation_pred
0,0.png,1,27797 2 28053 4 28063 2 28309 15 28565 19 2882...
1,1.png,1,3629 4 3883 8 4137 12 4390 17 4644 21 4898 25 ...
10,10.png,0,Healthy
100,100.png,1,19931 7 20187 10 20443 10 20699 10 20955 10 21...
101,101.png,1,6392 4 6645 7 6899 9 7150 15 7394 27 7645 32 7...
102,102.png,1,8475 1 8730 4 8986 5 9242 6 9498 7 9754 9 1001...
103,103.png,0,Healthy
104,104.png,0,Healthy
105,105.png,1,17528 1 17781 5 17846 1 18034 9 18102 2 18287 ...
106,106.png,1,23612 5 23866 9 24121 9 24377 6 29270 25 29524...


In [ ]:
# class CNNclassificationmodel(nn.Module):
#     def __init__(self, num_classes, in_channels, out_channels):
#         super(CNNclassificationmodel,self).__init__()
#         self.num_classes = num_classes
#         self.in_channels = in_channels
#         self.out_channels = out_channels

#         #Current size: 224x224 if input is  256*256
#         self.conv1 = nn.Sequential(
#             nn.Conv2d(in_channels, 16, kernel_size=3, stride=1, padding=1),
#             nn.BatchNorm2d(16),
#             nn.Dropout(p=0.1),
#             nn.ReLU(inplace=True),
#             nn.MaxPool2d(kernel_size=2, stride=2)
#         )

#         #Current size: 112x112  16*16 128*128
#         self.conv2 = nn.Sequential(
#             nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1),
#             nn.BatchNorm2d(32),
#             nn.Dropout(p=0.1),
#             nn.ReLU(inplace=True),
#             nn.MaxPool2d(kernel_size=2, stride=2)
#         )
  
#         # Current size: 56x56  8*8  64*64
#         self.conv3 = nn.Sequential(
#             nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
#             nn.BatchNorm2d(64),
#             nn.Dropout(p=0.1),
#             nn.ReLU(inplace=True),
#             nn.MaxPool2d(kernel_size=2, stride=2)
#         )

#         #Current size: 28x28 filter = 192 32*32
#         self.conv4 = nn.Sequential(
#             nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
#             nn.BatchNorm2d(128),
#             nn.Dropout(p=0.1),
#             nn.ReLU(inplace=True),
#             nn.MaxPool2d(kernel_size=2, stride=2)
#         )

#         # # #Current size: 14x14 filters = 128  16*16
#         self.conv5 = nn.Sequential(
#             nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
#             nn.BatchNorm2d(256),
#             nn.Dropout(p=0.1),
#             nn.ReLU(inplace=True),
#             nn.MaxPool2d(kernel_size=2, stride=2)
#         )

#         # 8*8
#         self.global_avg_pool = nn.AdaptiveAvgPool2d((1, 1))
#         self.fc = nn.Sequential(
#             nn.Linear(256*1*1, 128, bias=True),
#             nn.ReLU(inplace=True),
#             nn.Dropout(p=0.3),
#             nn.Linear(128, 32, bias=True),
#             nn.ReLU(inplace=True),
#             nn.Dropout(p=0.3),
#             nn.Linear(32, 1)
#         )
        

#     def forward(self, x):
#         x1 = self.conv1(x)
#         # x2 = self.conv2(x1)
#         x3 = self.conv3(x2)
#         x4 = self.conv4(x3)
#         x5 = self.conv5(x4)
#         x6 = self.global_avg_pool(x5)
#         x7 = x6.reshape(x.size(0), -1)
#         x8 = self.fc(x7)
#         return x8

In [117]:
class CustomDataset(Dataset):
    def __init__(self, images, masks, labels, is_train= True, resize_dim = (224, 224)):
        self.resize_dim = resize_dim
        self.images = images
        self.is_train = is_train
        self.image_transform = Compose([
            ToTensor(),
            Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225])
        ])
        self.label_transform = ToTensor()
        if is_train:
            self.masks = masks
            self.labels = labels
        else:
            self.masks = None
            self.labels = labels

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        if idx >= len(self.images):
            print('Reduce the index count as it is greater than the length of the dataset')
            return None
        if idx < 0:
            print('Index should be greater than or equal to 0')
            return None
        data = {}

        if self.is_train:
            data['image'] = Image.open(self.images[idx])
            data['mask'] = Image.open(self.masks[idx])
            data['class_label'] = self.labels.iloc[idx, 1]
            data['id'] = self.images[idx].split('/')[-1]
            data['image'], data['mask'] = joint_random_rotation(data['image'], data['mask'], 10)
            data['image'], data['mask'] = joint_random_horizontal_flip(data['image'], data['mask'])
            data['image'], data['mask'] = joint_random_vertical_flip(data['image'], data['mask'])
            data['image'], data['mask'] = joint_random_crop(data['image'], data['mask'], self.resize_dim)
            data['image'] = data['image'].resize(self.resize_dim)
            data['mask'] = data['mask'].resize(self.resize_dim)
            data['class_label'] = torch.tensor(data['class_label']).float()
            if self.image_transform:
                data['image'] = self.image_transform(data['image']) 
                data['mask'] = self.label_transform(data['mask'])
            return data
        else:
            data['image'] = Image.open(self.images[idx])
            data['id'] = self.images[idx].split('/')[-1]
            if self.image_transform:
                data['image'] = self.image_transform(data['image'])
            return data
    

In [118]:

# Split the images in val and train from train_path
def create_k_fold_split(train_path, train_csv_path):
    # Create a KFold object
    kf = KFold(n_splits=5, shuffle=True, random_state=42)

    train_img_paths = glob(os.path.join(train_path, 'images', '*.png'))
    train_img_paths.sort()
    train_mask_paths = glob(os.path.join(train_path, 'masks', '*.png'))
    train_mask_paths.sort()
    train_labels = pd.read_csv(train_csv_path)
    train_labels = train_labels.sort_values(by='id')
    train_labels.index = range(len(train_labels))
    split = list(kf.split(train_img_paths))
    return split, train_img_paths, train_mask_paths, train_labels


In [119]:
def get_datasets(train_img_paths, train_mask_paths, train_labels, val_img_paths, val_mask_paths, val_labels):
    train_dataset = CustomDataset(train_img_paths, train_mask_paths, train_labels, is_train=True, resize_dim=RESIZE_DIM)
    val_dataset = CustomDataset(val_img_paths, val_mask_paths, val_labels, is_train=True, resize_dim=RESIZE_DIM)
    return train_dataset, val_dataset

def get_dataloader(train_dataset, val_dataset, batch_size):
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    return train_loader, val_loader

def get_splitted_data(train_indices, val_indices):
    train = [train_img_paths[i] for i in train_indices]
    train_masks = [train_mask_paths[i] for i in train_indices]
    training_labels = train_labels.iloc[train_indices]
    val = [train_img_paths[i] for i in val_indices]
    val_masks = [train_mask_paths[i] for i in val_indices]
    val_labels = train_labels.iloc[val_indices]
    return train, train_masks, training_labels, val, val_masks, val_labels



split, train_img_paths, train_mask_paths, train_labels = create_k_fold_split(train_path, train_csv_path)

In [120]:
for repeat in range(1):
    for fold in fold_number:
        for lr in LEARNING_RATE:
            for batch in BATCH_SIZE:
                    weights = models.ResNet18_Weights
                    model = models.resnet18(weights=weights.IMAGENET1K_V1)
                    # num_ftrs = model.fc.in_features
                    model.fc = nn.Sequential(
                                nn.Linear(model.fc.in_features, 1)
                                    )
                    name = 'resnet18'
                    model = model.to(DEVICE)
                    # freeze all layers except the last one
                    for param in model.parameters():
                        param.requires_grad = False
                    for param in model.fc.parameters():
                        param.requires_grad = True
                    criterion = nn.BCEWithLogitsLoss(reduction='mean')
                    optimizer = optim.Adam(model.fc.parameters(), lr=lr, weight_decay=1e-5)
                    sum =summary (model, input_size=(batch, 3, RESIZE_DIM[0], RESIZE_DIM[1]), device=DEVICE)
                    train_indices, val_indices = split[fold]
                    with open(f'resnet18_model_summary_lr_{lr}_batch_{batch}.txt', 'w') as f:
                        f.write(str(sum))

                    train, train_masks, training_labels, val, val_masks, val_labels = get_splitted_data(train_indices, val_indices)
                    train_dataset, val_dataset = get_datasets(train, train_masks, training_labels, val, val_masks, val_labels)
                    train_loader, val_loader = get_dataloader(train_dataset, val_dataset, batch)      
                    best_model = None
                    best_val_loss = float('inf')
                    results = {
                        'train_loss': [],
                        'val_loss': [],
                        'train_accuracy': [],
                        'val_accuracy': [],
                        'val_precision': [],
                        'val_recall': [],
                        'val_f1_score': []
                    }
                    print(f'Training with learning rate: {lr}, batch size: {batch}, fold: {fold}, repeat: {repeat}')
                    for epoch in range(EPOCHS):
                        # Training Phase
                        model.train()
                        total_train_loss = 0
                        total_train_correct = 0
                        total_train_samples = 0
                        for data in train_loader:
                            image = data['image'].to(DEVICE)
                            label = data['class_label'].unsqueeze(1).float().to(DEVICE)
                            optimizer.zero_grad()
                            output = model(image)

                            loss = criterion(output, label)
                            loss.backward()
                            optimizer.step()

                            total_train_loss += loss.item() 
                            predicted = (torch.sigmoid(output) > 0.5).float()
                            total_train_correct += (predicted == label).float().sum().item()
                            total_train_samples += label.size(0)
                        # Compute average train loss and accuracy for the epoch
                        avg_train_loss = total_train_loss / len(train_loader)
                        train_accuracy = total_train_correct / total_train_samples
                        
                        results['train_loss'].append(avg_train_loss)
                        results['train_accuracy'].append(train_accuracy)
                        
                        # Validation Phase (every val_step epochs)
                        if epoch % val_step == 0:
                            model.eval()
                            total_val_loss = 0
                            tp = 0
                            fp = 0
                            fn = 0
                            tn = 0
                            with torch.no_grad():
                                for data in val_loader:
                                    image = data['image'].to(DEVICE)
                                    label = data['class_label'].unsqueeze(1).float().to(DEVICE)
                                    # Forward pass
                                    output = model(image)
                                    # Compute loss
                                    # print(f'validation output: {output}, val_label : {label}')
                                    loss = criterion(output, label)
                                    total_val_loss += loss.item()
                                    # Compute accuracy
                                    predicted = (torch.sigmoid(output) > 0.5).float()
                                    # print(f'predicted: {predicted}, val_label : {label}')
                                    tp += ((predicted == 1) & (label == 1)).sum().item()
                                    fp += ((predicted == 1) & (label == 0)).sum().item()
                                    fn += ((predicted == 0) & (label == 1)).sum().item()
                                    tn += ((predicted == 0) & (label == 0)).sum().item()
                                # Compute average validation loss and accuracy
                                avg_val_loss = total_val_loss / len(val_loader)
                                val_accuracy = (tp + tn) / (tp + fp + fn + tn)
                                val_precision = tp / (tp + fp + 1e-10)
                                val_recall = tp / (tp + fn + 1e-10)
                                val_f1_score = 2 * (val_precision * val_recall) / (val_precision + val_recall + 1e-10)
                                results['val_loss'].append(avg_val_loss)
                                results['val_accuracy'].append(val_accuracy)
                                results['val_recall'].append(val_recall)
                                results['val_f1_score'].append(val_f1_score)
                                results['val_precision'].append(val_precision)
                                # Save best model
                                if avg_val_loss < best_val_loss:
                                    best_val_loss = avg_val_loss
                                    best_model = model.state_dict()
                                    torch.save(best_model, f'classification_model/best_model_{name}_lr_{lr}_batch_{batch}_fold_{fold}_repeat_{repeat}.pth')
                                
                                #implement early stopping
                                if epoch > 50 and (avg_val_loss > np.mean(np.array(results['val_loss'][-20:]))):
                                    print(f"Early stopping at epoch {epoch+1} with validation loss {best_val_loss:.4f}")
                                    break
                                # Print epoch statistics
                                print(f'Epoch {epoch+1}:')
                                print(f'Train Loss: {avg_train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}')
                                print(f'Validation Loss: {avg_val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}')
                        # scheduler.step(avg_val_loss)           
                    # Save results to CSV
                    results_df = pd.DataFrame(results)
                    results_df.to_csv(f'class_results_{name}_lr_{lr}_batch_{batch}_fold_{fold}_repeat_{repeat+1}.csv', index=False)
                    print(f"Results saved for lr: {lr}, batch: {batch}, fold: {fold}, repeat: {repeat+1}")

Training with learning rate: 0.0001, batch size: 16, fold: 0, repeat: 0
Epoch 1:
Train Loss: 0.7129, Train Accuracy: 0.5502
Validation Loss: 0.6442, Validation Accuracy: 0.6319
Epoch 2:
Train Loss: 0.6610, Train Accuracy: 0.6135
Validation Loss: 0.5963, Validation Accuracy: 0.6978
Epoch 3:
Train Loss: 0.6122, Train Accuracy: 0.6616
Validation Loss: 0.5491, Validation Accuracy: 0.7253
Epoch 4:
Train Loss: 0.5759, Train Accuracy: 0.7125
Validation Loss: 0.5243, Validation Accuracy: 0.7912
Epoch 5:
Train Loss: 0.5401, Train Accuracy: 0.7414
Validation Loss: 0.4879, Validation Accuracy: 0.7967
Epoch 6:
Train Loss: 0.5087, Train Accuracy: 0.7937
Validation Loss: 0.4506, Validation Accuracy: 0.8462
Epoch 7:
Train Loss: 0.4867, Train Accuracy: 0.8116
Validation Loss: 0.4387, Validation Accuracy: 0.8626
Epoch 8:
Train Loss: 0.4645, Train Accuracy: 0.8501
Validation Loss: 0.4226, Validation Accuracy: 0.8956
Epoch 9:
Train Loss: 0.4355, Train Accuracy: 0.8638
Validation Loss: 0.3788, Validation 

In [121]:
test_img_paths = glob(os.path.join(test_path, 'images', '*.png'))
test_img_paths.sort()
test_data_csv = pd.read_csv(test_csv_path)
test_dataset = CustomDataset(test_img_paths, None, test_data_csv, is_train=False, resize_dim=RESIZE_DIM)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

In [75]:

model_path = r'/home/biomedialab/Desktop/Codes/Assignments/MLDS/Assignment_2/Q2/classification_model'
model_paths = glob(os.path.join(model_path, 'best_model_self_written*.pth'))
model_paths.sort()
print(f'Model Paths: {model_paths}')
y_pred = []

for model in model_paths:
    # Instantiate a new model instance (ensure the architecture is the same as during training)
    # Check if the model file corresponds to ResNet18 or CNNclassificationmodel
    model_instance = models.resnet18(pretrained=False)
    model_instance.fc = nn.Sequential(
        nn.Linear(model_instance.fc.in_features, 1)
    )
    
    # Load the model state dictionary
    model_state_dict = torch.load(model)
    model_instance.load_state_dict(model_state_dict)
    
    # Set the model to evaluation mode
    model_instance.eval()
    
    # List to store predictions for the current model
    model_preds = []
    
    # Iterate over the test DataLoader
    with torch.no_grad():
        for data in test_loader:
            images = data['image'].to(DEVICE)
            id = data['id']
            outputs = model_instance(images)
            probs = torch.sigmoid(outputs)
            probs = (probs > 0.5).int()
            model_preds.append((id[0],probs.cpu().item()))
    y_pred.append(list(model_preds))

test_data_df = pd.read_csv(test_csv_path)

y_pred_dict = {}
for model_preds in y_pred:
    for id, pred in model_preds:
        if id not in y_pred_dict:
            y_pred_dict[id] = [pred]
        else:
            y_pred_dict[id].append(pred)

# Perform majority voting
final_predictions = []
for id, preds in y_pred_dict.items():
    # Count the number of 1s and 0s
    count_1 = preds.count(1)
    count_0 = preds.count(0)
    
    # If the count of 1s is greater than the count of 0s, predict 1, else predict 0
    if count_1 > count_0:
        final_predictions.append((id, 1))
    else:
        final_predictions.append((id, 0))

# Convert the final predictions to a DataFrame
submission_df = pd.DataFrame(final_predictions, columns=['id', 'binary_pred'])
# Save the predictions to a CSV file
submission_df.to_csv('submission_class_resnet34.csv', index=False)

Model Paths: ['/home/biomedialab/Desktop/Codes/Assignments/MLDS/Assignment_2/Q2/classification_model/best_model_resnet34_lr_0.0001_batch_16_fold_0_repeat_0.pth', '/home/biomedialab/Desktop/Codes/Assignments/MLDS/Assignment_2/Q2/classification_model/best_model_resnet34_lr_0.0001_batch_16_fold_1_repeat_0.pth', '/home/biomedialab/Desktop/Codes/Assignments/MLDS/Assignment_2/Q2/classification_model/best_model_resnet34_lr_0.0001_batch_16_fold_2_repeat_0.pth', '/home/biomedialab/Desktop/Codes/Assignments/MLDS/Assignment_2/Q2/classification_model/best_model_resnet34_lr_0.0001_batch_16_fold_3_repeat_0.pth', '/home/biomedialab/Desktop/Codes/Assignments/MLDS/Assignment_2/Q2/classification_model/best_model_resnet34_lr_0.0001_batch_16_fold_4_repeat_0.pth']


/home/biomedialab/anaconda3/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/biomedialab/anaconda3/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
